In [1]:
import pickle as pkl
data_path = 'data.pkl'
with open(data_path, 'rb') as f:
    batch = pkl.load(f)

In [2]:
from models.CMA import LSTCWA, MultiDimAligner
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
D_TARGET   = 512
NUM_TOKENS = 64
WIN        = 64
STRIDE     = 32

aligner   = MultiDimAligner(D_TARGET).to(device)
compressor= LSTCWA(dim=D_TARGET, num_tokens=NUM_TOKENS,
                    win_size=WIN, stride=STRIDE).to(device)

# --------- 单 batch 推理示例 ---------
#compressed_batch = []
# 
# # 每张 WSI 输出一个 [L, D] token
tokens_per_backbone = []
for feats_list, coords_list, pad_list in zip(batch['imgs_list'],
                                            batch['coords_list'],
                                            batch['pad_mask_list']):
    """注意：
       - feats_list  : List[Tensor]，长度 = backbone 数
         每个 tensor 形状 [T_i, D_in_i]
       - coords_list : List[Tensor]，同上
       - pad_list    : List[BoolTensor]，同上
    """
    
    feats = feats.to(device)          # [T, D_in]
    coords= coords.to(device)
    pad   = pad.to(device)
    feats_ali = aligner(feats)        # → [T, D_TARGET]
    Z = compressor(feats_ali, coords, pad)  # [L, D_TARGET]
    tokens_per_backbone.append(Z)




In [25]:
for i in tokens_per_backbone:
    print(i.shape)

torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])


In [2]:
import os
import shutil

def delete_folders_by_keyword(root_path, keyword):
    """
    递归删除 root_path 下所有名称包含 keyword 的子文件夹
    :param root_path: 要搜索的根目录
    :param keyword: 要匹配的关键词（区分大小写）
    """
    for root, dirs, files in os.walk(root_path, topdown=False):
        for dir_name in dirs:
            if keyword in dir_name:
                folder_path = os.path.join(root, dir_name)
                try:
                    print(f"正在删除文件夹: {folder_path}")
                    shutil.rmtree(folder_path)  # 递归删除整个文件夹
                except Exception as e:
                    print(f"删除失败: {folder_path} - {e}")

delete_folders_by_keyword('/home/yuhaowang/project/CMA/CMA-MOE/output_2','moe')

正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/AIDPATH_GRADE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/DORID_6/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/BCNB_ALN/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/BRACS_COARSE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/SLNBREAST_SUBTYPE/moe
正在删除文件夹: /home/yuhaowang/project/CMA/CMA-MOE/output_2/TCGA-BRCA-SUBTYPE/moe
